#### Keras Tuner (hypertuning)

In [2]:
import tensorflow as tf
import keras_tuner as kt

Init Plugin
Init Graph Optimizer
Init Kernel


In [3]:
(img_train, label_train), (img_test, label_test) = tf.keras.datasets.fashion_mnist.load_data()
img_train = img_train / 255.0
img_test = img_test / 255.0

In [4]:
def model_builder(hp):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28))
    ])

    # 调整第一层的units单元数量，使其达到最优值。
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(tf.keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(tf.keras.layers.Dense(10))

    # 调整 optimizer 的 learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model


In [5]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Metal device set to: Apple M1


2021-08-12 21:52:22.573182: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-08-12 21:52:22.573455: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
# 与 model.fit() 一样
tuner.search(img_train, label_train,
             epochs=50, validation_split=0.2, callbacks=[early_stop])

Trial 30 Complete [00h 01m 02s]
val_accuracy: 0.8587499856948853

Best val_accuracy So Far: 0.890583336353302
Total elapsed time: 00h 16m 22s
INFO:tensorflow:Oracle triggered exit


In [7]:
best_hps = tuner.get_best_hyperparameters()[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 352 and the optimal learning rate for the optimizer
is 0.001.



#### 训练模型

使用从搜索中获得的超参数找到训练模型的最佳 epoch 数。

In [8]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc = history.history['val_accuracy']
best_epoch = val_acc.index(max(val_acc)) + 1

print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
  20/1500 [..............................] - ETA: 8s - loss: 1.3508 - accuracy: 0.5500  

2021-08-12 22:33:28.947253: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1490/1500 [============================>.] - ETA: 0s - loss: 0.4958 - accuracy: 0.8249

2021-08-12 22:33:36.436833: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1500/1500 [==============================] - 9s 6ms/step - loss: 0.4952 - accuracy: 0.8250 - val_loss: 0.4514 - val_accuracy: 0.8361
Epoch 2/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3689 - accuracy: 0.8656 - val_loss: 0.3509 - val_accuracy: 0.8744

使用最佳epoch重新训练

In [9]:
hyper_model = tuner.hypermodel.build(best_hps)

hyper_model.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/46
  20/1500 [..............................] - ETA: 8s - loss: 1.3901 - accuracy: 0.5172      

2021-08-12 22:42:46.252054: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1497/1500 [============================>.] - ETA: 0s - loss: 0.4963 - accuracy: 0.8248

2021-08-12 22:42:54.113462: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1500/1500 [==============================] - 9s 6ms/step - loss: 0.4961 - accuracy: 0.8249 - val_loss: 0.4270 - val_accuracy: 0.8478
Epoch 2/46
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3726 - accuracy: 0.8637 - val_loss: 0.4036 - val_accuracy: 0.8517

In [10]:
hyper_model.evaluate(img_test, label_test)

 20/313 [>.............................] - ETA: 1s - loss: 0.5084 - accuracy: 0.9047 

2021-08-12 22:51:05.470000: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 3s 9ms/step - loss: 0.5745 - accuracy: 0.8877

[0.5744982957839966, 0.8877000212669373]

验证是否真的是最优解
first Dense units: 352, optimizer lr: 0.001